In [1]:
#!/usr/bin/env python3
# install_submodules.py

import os
import subprocess
import sys
from pathlib import Path

def install_submodule(path):
    """安装单个子模块"""
    try:
        print(f"正在安装: {path}")
        result = subprocess.run([
            sys.executable, "-m", "pip", "install", "-e", str(path)
        ], capture_output=True, text=True, check=True)
        
        print(f"✅ 成功安装: {path}")
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ 安装失败: {path}")
        print(f"错误信息: {e.stderr}")
        return False

def find_submodules(root_dir="../"):
    """查找所有子模块目录"""
    submodules = []
    root_path = Path(root_dir)
    
    # 查找所有包含 setup.py 或 pyproject.toml 的子目录
    for item in root_path.iterdir():
        if item.is_dir() and not item.name.startswith('.'):
            has_setup_py = (item / "setup.py").exists()
            has_pyproject = (item / "pyproject.toml").exists()
            if has_setup_py or has_pyproject:
                submodules.append(item)
    
    return submodules

def main():
    """主函数"""
    print("🔍 搜索子模块...")
    submodules = find_submodules()
    
    if not submodules:
        print("❌ 未找到任何子模块")
        return
    
    print(f"📦 找到 {len(submodules)} 个子模块:")
    for module in submodules:
        print(f"  - {module.name}")
    
    print("\n🚀 开始安装...")
    success_count = 0
    
    for module in submodules:
        if install_submodule(module):
            success_count += 1
        print("-" * 50)
    
    print(f"\n📊 安装完成: {success_count}/{len(submodules)} 成功")

main()    


🔍 搜索子模块...
📦 找到 7 个子模块:
  - vnpy_ctastrategy
  - vnpy_paperaccount
  - vnpy_mysql
  - vnpy
  - vnpy_akshare
  - vnpy_ctabacktester
  - akshare

🚀 开始安装...
正在安装: ../vnpy_ctastrategy
✅ 成功安装: ../vnpy_ctastrategy
--------------------------------------------------
正在安装: ../vnpy_paperaccount
✅ 成功安装: ../vnpy_paperaccount
--------------------------------------------------
正在安装: ../vnpy_mysql
✅ 成功安装: ../vnpy_mysql
--------------------------------------------------
正在安装: ../vnpy
✅ 成功安装: ../vnpy
--------------------------------------------------
正在安装: ../vnpy_akshare
✅ 成功安装: ../vnpy_akshare
--------------------------------------------------
正在安装: ../vnpy_ctabacktester
✅ 成功安装: ../vnpy_ctabacktester
--------------------------------------------------
正在安装: ../akshare
✅ 成功安装: ../akshare
--------------------------------------------------

📊 安装完成: 7/7 成功


In [2]:
import mysql.connector
import pandas as pd


def create_connection():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            port=3306,
            user='root',
            password='123456',
            database='ASTOCK'
        )
        return connection
    except mysql.connector.Error as e:
        print(f"连接MySQL失败: {e}")
        return None

In [3]:
def get_min_market_value(year, month) -> list:
    # 获取市值数据
    market_values = get_market_values(year, month)

    # 按市值排序，选择前N个标的
    top_n = 10
    sorted_symbols = sorted(market_values, key=lambda x: x["market_value"])[:top_n]
    return sorted_symbols

def get_market_values(year, month) -> list:
    connection = create_connection()
    if not connection:
        return []
    
    finance_report_months = ['12', '03', '06', '09']
    year_of_interest = year - 1 if month < 4 else year
    report_month_of_interest = str(year_of_interest) + "-" + finance_report_months[(month - 1) // 3]

    cursor = connection.cursor()
    sql = f'''SELECT 
            a.symbol, a.name, a.update_time, (a.total_shares * b.close_price) AS market_value
            FROM finance a
            JOIN (
                SELECT 
                    symbol,
                    datetime,
                    close_price,
                    ROW_NUMBER() OVER (
                        PARTITION BY symbol, YEAR(datetime), MONTH(datetime) 
                        ORDER BY datetime DESC
                    ) as rn
                FROM daily
                WHERE exchange = 'SZSE'
                    AND datetime like '{report_month_of_interest}-%'
            ) b 
            ON a.symbol = b.symbol 
                AND YEAR(a.update_time) = YEAR(b.datetime)
                AND MONTH(a.update_time) = MONTH(b.datetime)
                AND b.rn = 1
            WHERE a.market = '深市'
                AND a.update_time like '{report_month_of_interest}-%'
                AND a.symbol like '00%'
                AND a.name not like '%ST%'
                AND a.name not like '%退'
            ;'''
    # print(f"SQL查询语句: {sql}")
    cursor.execute(sql)
    rows = cursor.fetchall()
    cursor.close()
    connection.close()

    # 将查询结果转换为字典列表
    values = [{"symbol": row[0], "name": row[1], "market_value": row[3]} for row in rows]
    return values


In [ ]:
from vnpy.trader.optimize import OptimizationSetting
from vnpy_ctastrategy.backtesting import BacktestingEngine
from datetime import datetime
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
from strategies.monthly_min_market_value_strategy import MonthlyMinMarketValueStrategy

goods_symbols = []

# end_pos = len(all_symbols)
end_pos = 10
start_day = datetime(2024, 4, 1)
end_day = datetime(2025, 3, 31)

# 初始化列表
month_first_list = []
# current_date = start_day.replace(day=1)  # 确保从每月的1日开始
current_date = start_day

while current_date <= end_day:
    month_first_list.append(current_date)

    if current_date.month == 12:
        current_date = current_date.replace(year=current_date.year + 1, month=1, day=1)
    else:
        current_date = current_date.replace(month=current_date.month + 1, day=1)

# 输出结果
print(month_first_list)

capital = 1000000
total_profits = list()
for i in range(len(month_first_list)):
    if i > 0:
        capital += total_profits[i - 1] / 10  # 累加上个月的总净利润
    start = month_first_list[i]
    end = month_first_list[i + 1] if i + 1 < len(month_first_list) else end_day
    print(f"开始日期: {start}")
    
    min_market_symbols = get_min_market_value(start.year, start.month)

    # 按市值排序，选择前N个标的
    symbols_candidates = min_market_symbols
    month_profits = list() 
    month_profit = 0
    print(f"候选标的：{symbols_candidates}")
    for symbol in symbols_candidates:
        engine = BacktestingEngine()
        engine.set_parameters(
            vt_symbol=symbol["symbol"] + ".SZSE",
            interval="d",
            start=start,
            end=end,
            rate=0.3/10000,
            slippage=0.2,
            size=100,
            pricetick=0.2,
            capital=capital,
        )
        engine.add_strategy(MonthlyMinMarketValueStrategy, {})

        engine.load_data()
        engine.run_backtesting()
        df = engine.calculate_result()
        res = engine.calculate_statistics()
        print(f"策略回测成功：{symbol['symbol']}，总净利润：{res['total_net_pnl']}")
        month_profits.append(res["total_net_pnl"])
        if res["total_net_pnl"] > 0:
            goods_symbols.append({"symbol": symbol["symbol"], "total_net_pnl": res["total_net_pnl"]})
            # print(f"策略回测成功：{symbol}，总净利润：{res['total_net_pnl']}")
    month_profit = sum(month_profits)
    print(f"{start.year}-{start.month}总净利润：{month_profit}")
    total_profits.append(month_profit)

print(f"总净利润：{sum(total_profits)}")


# print(f'goods_symbols: {goods_symbols}')

    # df.
    # engine.show_chart()

[datetime.datetime(2024, 4, 1, 0, 0), datetime.datetime(2024, 5, 1, 0, 0), datetime.datetime(2024, 6, 1, 0, 0), datetime.datetime(2024, 7, 1, 0, 0), datetime.datetime(2024, 8, 1, 0, 0), datetime.datetime(2024, 9, 1, 0, 0), datetime.datetime(2024, 10, 1, 0, 0), datetime.datetime(2024, 11, 1, 0, 0), datetime.datetime(2024, 12, 1, 0, 0), datetime.datetime(2025, 1, 1, 0, 0), datetime.datetime(2025, 2, 1, 0, 0), datetime.datetime(2025, 3, 1, 0, 0)]
开始日期: 2024-04-01 00:00:00
候选标的：[{'symbol': '002856', 'name': '美芝股份', 'market_value': 1165043208.0}, {'symbol': '002633', 'name': '申科股份', 'market_value': 1219500000.0}, {'symbol': '002193', 'name': '如意集团', 'market_value': 1219594472.32}, {'symbol': '002629', 'name': '仁智股份', 'market_value': 1231347360.0}, {'symbol': '001211', 'name': '双枪科技', 'market_value': 1250640000.0}, {'symbol': '002652', 'name': '扬子新材', 'market_value': 1341607680.0}, {'symbol': '000692', 'name': '惠天热电', 'market_value': 1406679014.4}, {'symbol': '000953', 'name': '河化股份', 'marke

In [ ]:
setting = OptimizationSetting()
setting.set_target("sharpe_ratio")
setting.add_parameter("atr_length", 25, 27, 1)
setting.add_parameter("atr_ma_length", 10, 30, 10)

engine.run_ga_optimization(setting)

In [ ]:
engine.run_bf_optimization(setting)